# Get required packages

In [50]:
%%capture
!pip install -r requirements.txt

# Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import pyogrio

# Read data

In [ ]:
sopp = pd.read_csv('data/ca_san_diego_2020_04_01.csv')
svi_tab = pd.read_csv('data/California.csv')

# These are the columns for merging later
svi_tab = svi_tab[['FIPS', 'RPL_THEMES']].copy()
svi_tab['FIPS'] = svi_tab['FIPS'].astype(str).str.replace(r'\.0$', '', regex=True) # chat regex

print(sopp[['date','service_area','search_conducted']].head())
print(svi_tab.head())

         date service_area  search_conducted
0  2014-01-01          110             False
1  2014-01-01          320             False
2  2014-01-01          320             False
3  2014-01-01          610             False
4  2014-01-01          930             False
         FIPS  RPL_THEMES
0  6001400100      0.1598
1  6001400200      0.1827
2  6001400300      0.2366
3  6001400400      0.1451
4  6001400500      0.2148


In [ ]:
# Shape files for tract SVI data and police service zones
cdc_gdb_path = "/Users/asmit/Downloads/SVI2014_CALIFORNIA_tract.gdb/"
beats_path = "./data/Police_Beats.geojson"

# Some info about the police service zones
# Contains meta data + spatial geometry
beats = gpd.read_file(beats_path)
print(beats.head())
print(beats.columns.tolist())
print(beats.crs)

   objectid  beat  div  serv        name  \
0         9   935    9   930  NORTH CITY   
1        13     0    0     0   SAN DIEGO   
2        14   511    5   510         NaN   
3        15   722    7   720      NESTOR   
4        16   314    3   310    BIRDLAND   

                                            geometry  
0  POLYGON ((-117.23876 32.98575, -117.2387 32.98...  
1  MULTIPOLYGON (((-117.22526 32.70267, -117.2252...  
2  MULTIPOLYGON (((-117.22529 32.7026, -117.22525...  
3  POLYGON ((-117.09042 32.58382, -117.09001 32.5...  
4  POLYGON ((-117.15149 32.8065, -117.1514 32.806...  
['objectid', 'beat', 'div', 'serv', 'name', 'geometry']
EPSG:4326


In [ ]:
# This is just to see which part of the SVI folder has the data
print(pyogrio.list_layers(cdc_gdb_path))

[['CALIFORNIA_tract' 'MultiPolygon Z']]


/Users/asmit/Documents/SDS357/venv/lib/python3.13/site-packages/pyogrio/core.py:129: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D MultiPolygon' is converted to 'MultiPolygon Z'
  return ogr_list_layers(get_vsi_path_or_buffer(path_or_buffer))


In [ ]:
# Look at the SVI tracts now
layer_name = "CALIFORNIA_tract"

tracts = gpd.read_file(cdc_gdb_path, layer=layer_name)

# Also contains geometry
print(tracts.head())
print(tracts.columns.tolist())
print(tracts.crs)

/Users/asmit/Documents/SDS357/venv/lib/python3.13/site-packages/pyogrio/raw.py:200: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D MultiPolygon' is converted to 'MultiPolygon Z'
  return ogr_read(


               AFFGEOID TRACTCE  ST        STATE ST_ABBR STCNTY    COUNTY  \
0  1400000US06001400100  400100  06   California      CA  06001   Alameda   
1  1400000US06001400200  400200  06   California      CA  06001   Alameda   
2  1400000US06001400300  400300  06   California      CA  06001   Alameda   
3  1400000US06001400400  400400  06   California      CA  06001   Alameda   
4  1400000US06001400500  400500  06   California      CA  06001   Alameda   

          FIPS                                       LOCATION  AREA_SQMI  ...  \
0  06001400100  Census Tract 4001, Alameda County, California   2.661917  ...   
1  06001400200  Census Tract 4002, Alameda County, California   0.226817  ...   
2  06001400300  Census Tract 4003, Alameda County, California   0.426770  ...   
3  06001400400  Census Tract 4004, Alameda County, California   0.275958  ...   
4  06001400500  Census Tract 4005, Alameda County, California   0.227919  ...   

   F_THEME4  F_TOTAL  E_UNINSUR  M_UNINSUR  EP_UNI

In [ ]:
# Match data types and string conventions etc
# from chat
tracts['FIPS'] = tracts['FIPS'].astype(str).str.replace(r'\.0$', '', regex=True)
tracts_sd = tracts[tracts['FIPS'].str.startswith('06073')].copy()

# Keep the geometry and SVI (single number) and the FIPS code
tracts_sd = tracts_sd[['FIPS', 'RPL_THEMES', 'geometry']].copy()

print(tracts_sd[['FIPS', 'RPL_THEMES']].head())
print("Rows in San Diego tracts:", len(tracts_sd))

             FIPS  RPL_THEMES
4931  06073008509      0.4912
4932  06073008510      0.4610
4933  06073008511      0.4802
4934  06073008512      0.1664
4935  06073008513      0.0467
Rows in San Diego tracts: 627


Now match between the police zones and census tracts. Suppose that we are in a space where san diego is the whole space.

The set $A$ is the set of policing zones (it is made up of some smaller sets, $A_j$, corresponding to each zone in particular).

The set $B$ is the set of all census tracts (also made up of sets $B_j$). 

$A$ and $B$ are both subsets of the whole space (san diego greater area I think, this part was from chat). We want to match the data where $A \cap B$ i.e. where the police zones intersect with the census tracts on social vulnerability.

That is what this block of code does. There is some modification of the SVI data (suggested by chat) so that, when taking the average, it is weighted. For example, when two tracts with different SVI intersect one police zone, we take a weighted average, assigning higher weights to tracts that intersect more heavily with the police zone.

In [37]:
# Make CRS match
if beats.crs != tracts_sd.crs:
    tracts_sd = tracts_sd.to_crs(beats.crs)

# Project to a metric CRS for area calculations
beats_proj = beats.to_crs(epsg=26911)      # UTM Zone 11N
tracts_proj = tracts_sd.to_crs(epsg=26911)

# Spatial intersection
inter = gpd.overlay(beats_proj, tracts_proj, how="intersection")

# Area-weighted SVI
inter["overlap_area"] = inter.geometry.area
inter["weighted_svi"] = inter["RPL_THEMES"] * inter["overlap_area"]

print(inter.head())

   objectid  beat  div  serv        name         FIPS  RPL_THEMES  \
0         9   935    9   930  NORTH CITY  06073017029      0.0102   
1         9   935    9   930  NORTH CITY  06073017106      0.1011   
2         9   935    9   930  NORTH CITY  06073017304      0.3095   
3         9   935    9   930  NORTH CITY  06073017306      0.0422   
4         9   935    9   930  NORTH CITY  06073008324      0.0189   

                                            geometry  overlap_area  \
0  MULTIPOLYGON Z (((479103.087 3649461.789 0, 47...  2.490648e+06   
1  MULTIPOLYGON Z (((478835.266 3649610.423 0, 47...  1.821234e+04   
2  POLYGON Z ((476337.773 3649093.502 0, 476337.4...  5.891610e+04   
3  MULTIPOLYGON Z (((477697.717 3649731.986 0, 47...  6.566568e+05   
4  POLYGON Z ((476212.978 3649065.552 0, 476210.0...  2.955893e+02   

   weighted_svi  
0  25404.609253  
1   1841.267895  
2  18234.534197  
3  27710.918559  
4      5.586638  


Now, we actually do this weighted averaging

In [ ]:
beat_col = "serv"

svc_svi = (
    inter.groupby(beat_col, dropna=False)
    .agg(
        weighted_svi_sum=("weighted_svi", "sum"),
        overlap_area_sum=("overlap_area", "sum")
    )
    .reset_index()
)

svc_svi["svi_rpl_themes"] = svc_svi["weighted_svi_sum"] / svc_svi["overlap_area_sum"]

svc_svi = svc_svi[[beat_col, "svi_rpl_themes"]]

print(svc_svi.head())

   serv  svi_rpl_themes
0     0        0.029338
1   110        0.241848
2   120        0.143232
3   230        0.162551
4   240        0.131157


Finally, we combine into one data set by merging with the SOPP data. It is a left join, so that all of the service area columns present in the SOPP data set are still there (in case we want to work with those?).

In [40]:
sopp = pd.read_csv("./data/sopp.csv")  # or your full file path
sopp["service_area"] = sopp["service_area"].astype(str)

svc_svi[beat_col] = svc_svi[beat_col].astype(str)

sopp_final = sopp.merge(
    svc_svi,
    left_on="service_area",
    right_on=beat_col,
    how="left"
)

print(sopp_final[["date", "service_area", "search_conducted", "svi_rpl_themes"]].head())
print("Missing SVI rows:", sopp_final["svi_rpl_themes"].isna().sum())

         date service_area  search_conducted  svi_rpl_themes
0  2014-01-01          110             False        0.241848
1  2014-01-01          320             False        0.213643
2  2014-01-01          320             False        0.213643
3  2014-01-01          610             False        0.121181
4  2014-01-01          930             False        0.075382
Missing SVI rows: 11627


In [49]:
sopp_final.to_csv('./data/sopp_svi_merged.csv')

Sanity checks on the resulting data set (from chat). The next couple of chunks all do this so that I don't have to scroll when reading it all.

In [54]:
df = sopp_final.copy()

print("Shape:", df.shape)
print("\nColumns:")
print(df.columns.tolist())
print()

# Missing data
print("\n=== SVI missingness ===")
print(df["svi_rpl_themes"].isna().value_counts(dropna=False))
print("Missing SVI proportion:", df["svi_rpl_themes"].isna().mean())

Shape: (383027, 23)

Columns:
['raw_row_number', 'date', 'time', 'service_area', 'subject_age', 'subject_race', 'subject_sex', 'type', 'arrest_made', 'citation_issued', 'warning_issued', 'outcome', 'contraband_found', 'search_conducted', 'search_person', 'search_vehicle', 'search_basis', 'reason_for_search', 'reason_for_stop', 'raw_action_taken', 'raw_subject_race_description', 'serv', 'svi_rpl_themes']


=== SVI missingness ===
svi_rpl_themes
False    371400
True      11627
Name: count, dtype: int64
Missing SVI proportion: 0.030355562401606154


The above tells us that most of the data is there and it looks like the join was (relatively) successful. The small amount of missing data (I think) makes sense since some of the police zones were weird anyway

In [56]:
# SVI ranges (since chat did weighted averaging)
print("\n=== SVI range check ===")
print(df["svi_rpl_themes"].describe())

bad_range = df[(df["svi_rpl_themes"] < 0) | (df["svi_rpl_themes"] > 1)]
print("Rows with SVI outside [0,1]:", len(bad_range))
if len(bad_range) > 0:
    print(bad_range[["service_area", "svi_rpl_themes"]].head())

# Checking that SVI is consistent 
print("\n=== Within-service_area SVI consistency ===")

svc_check = (
    df.groupby("service_area")["svi_rpl_themes"]
      .nunique(dropna=True)
      .reset_index(name="n_unique_svi")
)

print(svc_check["n_unique_svi"].value_counts().sort_index())

bad_services = svc_check[svc_check["n_unique_svi"] > 1]
print("Service areas with >1 SVI value:", len(bad_services))
if len(bad_services) > 0:
    print(bad_services.head())

print("\n=== Merge coverage by service_area ===")

svc_missing = (
    df.groupby("service_area")["svi_rpl_themes"]
      .apply(lambda x: x.isna().mean())
      .reset_index(name="prop_missing_svi")
      .sort_values("prop_missing_svi", ascending=False)
)

print(svc_missing.head(10))


=== SVI range check ===
count    371400.000000
mean          0.346642
std           0.232837
min           0.075382
25%           0.143232
50%           0.241848
75%           0.549244
max           0.849382
Name: svi_rpl_themes, dtype: float64
Rows with SVI outside [0,1]: 0

=== Within-service_area SVI consistency ===
n_unique_svi
0     6
1    19
Name: count, dtype: int64
Service areas with >1 SVI value: 0

=== Merge coverage by service_area ===
   service_area  prop_missing_svi
24      Unknown               1.0
2           130               1.0
23       County               1.0
22     Bulletin               1.0
20          840               1.0
14          630               1.0
13          620               0.0
21          930               0.0
19          830               0.0
18          820               0.0


It seems like most police zones have a unique SVI. As you can see, some were messed up (Bulletin County is probably a full county name that somehow got separated, it seems like a pain in the ass to fix manually, so it's probably fine).

Also, all of the SVI's are in the correct interval, $[0, 1]$.

In [57]:
print("\n=== Key variable missingness ===")
key_cols = ["search_conducted", "subject_age", "subject_race", "subject_sex", "reason_for_stop", "service_area", "svi_rpl_themes"]
print(df[key_cols].isna().mean().sort_values(ascending=False))

print("\n=== search_conducted distribution ===")
print(df["search_conducted"].value_counts(dropna=False))


=== Key variable missingness ===
subject_age         0.031233
svi_rpl_themes      0.030356
subject_race        0.003222
subject_sex         0.001726
reason_for_stop     0.000572
search_conducted    0.000000
service_area        0.000000
dtype: float64

=== search_conducted distribution ===
search_conducted
False    366739
True      16288
Name: count, dtype: int64


Our regular variables of interest also seem fine.

In [ ]:
tmp = df.dropna(subset=["svi_rpl_themes", "search_conducted"]).copy()

if tmp["search_conducted"].dtype == object:
    tmp["search_conducted"] = tmp["search_conducted"].astype(str).str.lower().map({
        "true": 1, "false": 0, "1": 1, "0": 0
    })
else:
    tmp["search_conducted"] = tmp["search_conducted"].astype(int)

tmp = tmp.dropna(subset=["search_conducted"])

tmp["svi_quartile"] = pd.qcut(tmp["svi_rpl_themes"], 4, labels=["Q1(low)", "Q2", "Q3", "Q4(high)"])

summary = (
    tmp.groupby("svi_quartile")
       .agg(
           n=("search_conducted", "size"),
           search_rate=("search_conducted", "mean"),
           svi_min=("svi_rpl_themes", "min"),
           svi_max=("svi_rpl_themes", "max"),
       )
       .reset_index()
)

print("\n=== Search rate by SVI quartile ===")
print(summary)


=== Search rate by SVI quartile ===
  svi_quartile       n  search_rate   svi_min   svi_max
0      Q1(low)  100062     0.028852  0.075382  0.143232
1           Q2  106168     0.029142  0.162551  0.241848
2           Q3   78279     0.055366  0.321448  0.549244
3     Q4(high)   86891     0.064989  0.653392  0.849382


This is just some intro analysis (from chat). It looks like we can see a general trend of high SVI rows having a higher search rate. (0.02 for Q1 as opposed to 0.06 for Q4).